In [2]:
import csv
import cv2
import numpy as np

# after copying and extracting the data on EC2 instance
# We want read the data
# for each line extratc the path to the camera image path
# but we need to update the path cause it's not on out local machin
# anymore, and it's on the AWS instance.
lines = []
with open ('../data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
images = []
measurements = []
# easy way to update the path:
# Split the path on it's slashes and then extract the final token
# which will be file name.
# then we can add that file name to the end of the image directory
# here on the AWS instance.
# Now that we have the current path we can use opencv to load the image
# and append it to the our list of images
# we can do the same thing for the steering measurements, which will serve 
# as our output labels, but with this difference that there would be no path
# or no images to handle, just simply extract the 4th token from the csv line
for line in lines:
        source_path = line[0]
        filename = source_path.split('/')[-1]
        current_path = '../data/IMG/' + filename
        image = cv2.imread(current_path)
        images.append(image)
        measurement = float(line[3])
        measurements.append(measurement)

# Now we need to format them in numpy arrays cause that's Keras requirment
X_train = np.array(images)
y_train = np.array(measurements)
        
        

FileNotFoundError: [Errno 2] No such file or directory: '../data/driving_log.csv'

Now we are going to build the most basic NW possible
Just to verify that everthing is working fine.
This NW is going to be a flattened image connected to a
single output node. This single output node is going to predict 
the steering angle, which makes this a regression network.

For a Classification Netwrok we might need to apply a softmax activation function to the output layer. But for a regression network like this we just need a single output node to directly predict the steering angle for us.

In [1]:
from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()
model.add(Flatten(input_shape=(160, 320, 3)))
model.add(Dense(1))

# For the loss function we use mean square , or MSE, which is differnt
# than cross_entropy function, again becuase this is a regression network 
# not a classifer network.

# We want to minimize the error between the stearing measurementss that the 
# Network predicts and the ground truth steering measurement.
# MSE is good loss function for this.

model.compile(loss='mse', optimizer='adam')

# Once the model is compiled we'll train the feature and label arrays
# we just build.
# WE also shuffle the data and also split off 20% of the data to use 
# for validation set.

model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epochs=7)

# Now we will save the model and download it onto our local machine,
# and see if it works for driving the simulator.

model.save('model.h5')



Using TensorFlow backend.


NameError: name 'X_train' is not defined

By default Keras trains for 10 epochs. But we can see the validation loss decreases for the first 7 epochs and then it starts to climbing again.
This is a sign that maybe we are over fitting the data. The validation loss should decrease for almost all the epochs.

Next step is to download this model toour local machine.